In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("phish.csv")

In [3]:
import pandas as pd
from urllib.parse import urlparse

def preprocess_url(url):

    parsed_url = urlparse(url)

    domain = parsed_url.netloc
    subdomain = parsed_url.hostname.split('.')[:-2] 
    path = parsed_url.path
    query = parsed_url.query

    features = {
        'id': None,
        'NumDots': domain.count('.'),
        'SubdomainLevel': len(subdomain),
        'PathLevel': path.count('/') - 1,  
        'UrlLength': len(url),
        'NumDash': domain.count('-'),
        'NumDashInHostname': parsed_url.hostname.count('-'),
        'AtSymbol': '@' in domain,
        'TildeSymbol': '~' in url,
        'NumUnderscore': domain.count('_'),
        'NumPercent': url.count('%'),
        'NumQueryComponents': len(query.split('&')) if query else 0,
        'NumAmpersand': url.count('&'),
        'NumHash': url.count('#'),
        'NumNumericChars': sum(char.isdigit() for char in url),
        'NoHttps': not url.startswith('https://'),
        'RandomString': None,
        'IpAddress': None,
        'DomainInSubdomains': domain in subdomain,
        'DomainInPaths': domain in path,
        'HttpsInHostname': 'https' in parsed_url.hostname,
        'HostnameLength': len(parsed_url.hostname),
        'PathLength': len(path),
        'QueryLength': len(query),
        'DoubleSlashInPath': '//' in path,
        'CLASS_LABEL': None
    }

    return pd.DataFrame([features])

url = "https://www.scopus.com/record/display.uri?eid=2-s2.0-84977110651&origin=resultslist&sort=cp-f&src=s&st1=phishing&nlo=&nlr=&nls=&sid=1557df7d46497bd4ad2787bc56bf1291&sot=b&sdt=cl&cluster=scopubyr%2c%222019%22%2ct%2c%222018%22%2ct%2c%222017%22%2ct%2c%222016%22%2ct%2c%222015%22%2ct%2c%222014%22%2ct&sl=23&s=TITLE-ABS-KEY%28phishing%29&relpos=0&citeCnt=334&searchTerm="
preprocessed_data = preprocess_url(url)

du = pd.DataFrame(preprocessed_data)
print(du)


     id  NumDots  SubdomainLevel  PathLevel  UrlLength  NumDash  \
0  None        2               1          1        365        0   

   NumDashInHostname  AtSymbol  TildeSymbol  NumUnderscore  ...  RandomString  \
0                  0     False        False              0  ...          None   

   IpAddress  DomainInSubdomains  DomainInPaths  HttpsInHostname  \
0       None               False          False            False   

   HostnameLength PathLength QueryLength  DoubleSlashInPath  CLASS_LABEL  
0              14         19         323              False         None  

[1 rows x 26 columns]


In [4]:
df.head()

,id,NumDots,SubdomainLevel,PathLevel,UrlLength,NumDash,NumDashInHostname,AtSymbol,TildeSymbol,NumUnderscore,...,RandomString,IpAddress,DomainInSubdomains,DomainInPaths,HttpsInHostname,HostnameLength,PathLength,QueryLength,DoubleSlashInPath,CLASS_LABEL
0,1,3,1,5,72,0,0,0,0,0,...,0,0,0,0,0,21,44,0,0,1
1,2,3,1,3,144,0,0,0,0,2,...,0,0,0,0,0,17,16,103,0,1
2,3,3,1,2,58,0,0,0,0,0,...,0,0,0,0,0,27,24,0,0,1
3,4,3,1,6,79,1,0,0,0,0,...,0,0,0,1,0,22,50,0,0,0
4,5,3,0,4,46,0,0,0,0,0,...,1,0,0,1,0,10,29,0,0,1


In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [6]:
df1=df.drop('CLASS_LABEL',axis=1)

In [7]:
df1.head()

,id,NumDots,SubdomainLevel,PathLevel,UrlLength,NumDash,NumDashInHostname,AtSymbol,TildeSymbol,NumUnderscore,...,NoHttps,RandomString,IpAddress,DomainInSubdomains,DomainInPaths,HttpsInHostname,HostnameLength,PathLength,QueryLength,DoubleSlashInPath
0,1,3,1,5,72,0,0,0,0,0,...,1,0,0,0,0,0,21,44,0,0
1,2,3,1,3,144,0,0,0,0,2,...,1,0,0,0,0,0,17,16,103,0
2,3,3,1,2,58,0,0,0,0,0,...,1,0,0,0,0,0,27,24,0,0
3,4,3,1,6,79,1,0,0,0,0,...,1,0,0,0,1,0,22,50,0,0
4,5,3,0,4,46,0,0,0,0,0,...,1,1,0,0,1,0,10,29,0,0


In [8]:
df2=df['CLASS_LABEL']

In [9]:
df2.head()

0    1
1    1
2    1
3    0
4    1
Name: CLASS_LABEL, dtype: int64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df1, df2, random_state = 0)

In [11]:
knn = KNeighborsClassifier(n_neighbors = 22)

In [12]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=22)

In [13]:
y_pred = knn.predict(X_test)

In [14]:
print(y_pred)

[0 1 1 ... 1 1 1]


In [15]:
from sklearn.metrics import accuracy_score
print("Accuracy with k=22", accuracy_score(y_test, y_pred)*100)


Accuracy with k=22 97.2


In [16]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer

du = du.fillna(0)
Xt = du.drop('CLASS_LABEL', axis=1)  
yt = du['CLASS_LABEL']  
y_pre = knn.predict(Xt)
if y_pre == 1:
    print("This URL could potentially be a phishing attempt, do check before you click on the link!")
else:
    print("It does look legit, you could open up the link, but again at your own risk.")


This URL could potentially be a phishing attempt, do check before you click on the link!
